In [13]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
from policyengine_us.system import system
import pandas as pd
import plotly.graph_objects as go
from policyengine_core.charts import format_fig
import numpy as np
import csv

In [14]:

eitc_reform = Reform.from_dict({
  "gov.irs.credits.eitc.max[0].amount": {
    "2026-01-01.2026-12-31": 1502,
    "2027-01-01.2027-12-31": 1532,
    "2028-01-01.2028-12-31": 1563,
    "2029-01-01.2029-12-31": 1594,
    "2030-01-01.2030-12-31": 1626,
    "2031-01-01.2031-12-31": 1658,
    "2032-01-01.2032-12-31": 1691,
    "2033-01-01.2033-12-31": 1724,
    "2034-01-01.2034-12-31": 1758,
    "2035-01-01.2035-12-31": 1793
  },
  "gov.irs.credits.eitc.eligibility.age.max": {
    "2026-01-01.2100-12-31": np.inf
  },
  "gov.irs.credits.eitc.eligibility.age.min": {
    "2026-01-01.2100-12-31": 19
  },
  "gov.irs.credits.eitc.phase_in_rate[0].amount": {
    "2026-01-01.2100-12-31": 0.153
  },
  "gov.irs.credits.eitc.phase_out.rate[0].amount": {
    "2026-01-01.2100-12-31": 0.153
  },
  "gov.irs.credits.eitc.phase_out.start[0].amount": {
    "2026-01-01.2026-12-31": 11610,
    "2027-01-01.2027-12-31": 11842,
    "2028-01-01.2028-12-31": 12080,
    "2029-01-01.2029-12-31": 12318,
    "2030-01-01.2030-12-31": 12562,
    "2031-01-01.2031-12-31": 12806,
    "2032-01-01.2032-12-31": 13063,
    "2033-01-01.2033-12-31": 13320,
    "2034-01-01.2034-12-31": 13584,
    "2035-01-01.2035-12-31": 13854,
  },
    "gov.irs.credits.eitc.eligibility.age.min_student": {
      "2026-01-01.2100-12-31": 24
    },
    "gov.simulation.labor_supply_responses.elasticities.income": {
        "2024-01-01.2100-12-31": -0.05
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.1": {
        "2024-01-01.2100-12-31": 0.31
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.2": {
        "2024-01-01.2100-12-31": 0.28
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.3": {
        "2024-01-01.2100-12-31": 0.27
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.4": {
        "2024-01-01.2100-12-31": 0.27
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.5": {
        "2024-01-01.2100-12-31": 0.25
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.6": {
        "2024-01-01.2100-12-31": 0.25
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.7": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.8": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.9": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.secondary": {
        "2024-01-01.2100-12-31": 0.27
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.10": {
        "2024-01-01.2100-12-31": 0.22
  },
}, country_id="us")


In [15]:
def calculate_total_budgetary_impact(reform, year):

    # For production code, use this line instead for the real dataset:
    # baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
    # For compatibility with the example:
    baseline = Microsimulation(dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")
    baseline_income_tax = baseline.calculate("income_tax", period=year).sum()

    reformed = Microsimulation(reform=reform, dataset="hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5")

    reformed_income_tax = reformed.calculate("income_tax", period=year).sum()

    impact = reformed_income_tax - baseline_income_tax
    return impact


In [16]:

def calculate_ten_year_projection(reform):
    results = []

    with open("eitc_reform_budgetary_impact.csv", "w", newline="") as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["Year", "Budgetary Impact"])

        for year in range(2026, 2036):
            print(f"Computing budgetary impact for year {year}...")
            impact = calculate_total_budgetary_impact(reform, year)
            results.append({"Year": year, "Budgetary Impact": impact})

            csvwriter.writerow([year, impact])
            print(f"Year {year} completed. Impact: ${impact/1e9:.2f} billion")

    print("All calculations complete. Results saved to 'eitc_reform_budgetary_impact.csv'")
    return pd.DataFrame(results)


In [17]:

def create_ten_year_bar_chart(df):
    # We invert the sign to show costs as positive numbers in the chart
    # This makes it more intuitive to read (higher bars = bigger costs)
    df["Budgetary Impact Billions"] = (-df["Budgetary Impact"] / 1e9).round(1)

    fig = go.Figure(
        go.Bar(
            x=df["Year"],
            y=df["Budgetary Impact Billions"],
            text=df["Budgetary Impact Billions"].apply(lambda x: f"${x:.1f}B"),
            textposition="auto",
            marker_color="#105293",
        )
    )

    fig.update_layout(
        title="10-Year Federal Budgetary Impact of the EITC Expansion",
        xaxis_title="Year",
        yaxis_title="Cost (Billions $)",
        xaxis=dict(tickmode="linear"),
        yaxis=dict(zeroline=True, zerolinewidth=2, zerolinecolor="Black"),
    )

    return format_fig(fig)


In [18]:

# Main execution
if __name__ == "__main__":
    print("Starting 10-year budgetary impact calculation for EITC expansion...")
    df_projection = calculate_ten_year_projection(eitc_reform)
    print("Calculation complete. Summary of results:")
    print(df_projection)


    print("Generating bar chart...")
    fig = create_ten_year_bar_chart(df_projection)
    fig.write_html("eitc_reform_impact_chart.html")
    print("Bar chart saved to 'eitc_reform_impact_chart.html'")
    
    # Optionally show the chart if running in an interactive environment
    fig.show()
    print("Analysis complete.")

Starting 10-year budgetary impact calculation for EITC expansion...
Computing budgetary impact for year 2026...
Year 2026 completed. Impact: $-17.49 billion
Computing budgetary impact for year 2027...
Year 2027 completed. Impact: $-17.46 billion
Computing budgetary impact for year 2028...


KeyboardInterrupt: 